In [1]:
# install boto3 for AWS connection
dbutils.library.installPyPI("boto3", version="1.9.157")
dbutils.library.restartPython()

PyPI package boto3 has been installed already. The previous installed package is PyPI:(boto3)-(1.9.157)-(empty)-(empty). To resolve this issue detach and retach the notebook to create a new environment or rename the package.

In [2]:
# install these Maven packages in the cluster
# JohnSnowLabs:spark-nlp:2.3.4

In [3]:
# import AWS credentials
# import config.py ##for local
%run "/dbfs/FileStore/tables/config" ##for databricks

In [4]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [5]:
# get or create Spark session
app_name = "spark-airbnb-sentiment"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [6]:
import boto3

secret_name = my_secret_name
region_name = my_region_name
access_key  = my_access_key
secret_key  = my_secret_key

session      = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)
client       = session.client('secretsmanager')
secret_value = client.get_secret_value(SecretId=secret_name)

In [7]:
import json
def get_connection(secret_value):
    return json.loads(secret_value['SecretString'])

In [8]:
connection = get_connection(secret_value)

# Postgres credentials
jdbcHostname = connection['host']
jdbcPort     = connection['port']
jdbcDatabase = "postgres"
dialect      = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user"     : jdbcUsername,
  "password" : jdbcPassword,
  "driver"   : "org.postgresql.Driver" 
}

In [9]:
# Read from reviews_full table

table = "reviews_full"

df = spark.read.jdbc(url=jdbcUrl, table=table, properties=connectionProperties)
df.printSchema()

root
-- listing_id: integer (nullable = true)
-- id: integer (nullable = true)
-- date: date (nullable = true)
-- reviewer_id: integer (nullable = true)
-- reviewer_name: string (nullable = true)
-- comments: string (nullable = true)

In [10]:
display(df.select(df.columns))

listing_id,id,date,reviewer_id,reviewer_name,comments
3862246,54663524,2015-11-22,985696,Lydia,"Denise was super friendly and her home was impeccably clean and beyond charming. Her two little dogs were quiet friendly and the separate entrance was very nice. Have never seen such a well stocked bathroom, an keurig right in the bedroom, and excellent water pressure! A great representation and slice of Austin. Would highly recommend staying with her!"
3862246,56898680,2015-12-19,17020551,Leslie,"Denise was a very sweet, accommodating host. Great location, clean space, everything was as described."
3862246,58253716,2016-01-01,21499045,Jackson,Couldn't have been more welcoming or serious about her guest experience. Great thought and amenities provided for stay. Wouldn't hesitate to book with again.
3862246,58445874,2016-01-02,8313146,Jaime,"I just loved my stay at Denise's home! And, I LOVE her decor style! Everything thing one could think of to have at a BNB was available. It was also impeccably clean with a private room and bath. Her dogs are awesome too! I would definitely stay there again. I highly recommend her property. It is also a very short drive to Downtown. My stay there was just fantastic."
3862246,58584255,2016-01-03,2806995,Rachel,"Denise was very welcoming and made us feel right at home. She gave us full access to the kitchen, and we really appreciated all of the things she does to go above and beyond as a host. The bathroom was fully stocked with nice shampoos/soaps/amenities, and the bedroom was very comfortable. The room has a private entrance so that we could come and go as we pleased. Her dogs were adorable. All in all we really felt at home at Denise's house and would love to return."
3862246,59372905,2016-01-10,48798728,Charlotte,"I couldn't have asked for a warmer welcome, or for a more comfortable stay. Denise is a very thoughtful and committed host, who will always go the extra mile for guests. Whatever the question, she will endeavour to answer it. Her care for detail is reflected in the space she has provided: spotless and stylish, with plenty of character. She also strikes me as a very versatile host. I was in Austin for a conference, and was able to concentrate well at Denise's place when (unfortunately) I needed to work. But her sense of fun will also make her an excellent guide if you are out to discover all the cool things the city has to offer! So - if you're going to Austin, stay here!"
3862246,60522554,2016-01-24,26792421,Rubie,"Denise was SO nice and welcoming! :) She stayed in contact with us as we traveled and made it easy to check in with her and get settled. She went out of her way to make our stay there enjoyable in many little ways. She provided some essentials that other airBNB's don't like spare toothbrushes, lotion, etc. she gave a great recommendation for lunch too! The room is very clean and comfortable and you really can have as much privacy as you want with your own separate entrance, but Denise was also super friendly and willing to help in any way. Overall we had a great trip!"
3862246,61017340,2016-01-30,30715591,Christi,The host canceled this reservation the day before arrival. This is an automated posting.
3862246,66008610,2016-03-18,15770643,Val,Peter was helpful in arranging an earlier check-in time so I could drop my bags off before heading off to SXSW. He showed me around and the backyard was awesome - I was bummed to not be able to spend more time there and sit out in the yard. The neighborhood is quite but a short ride away from downtown. The room is perfect if you're going to be out and about exploring. Thanks for everything.
3862246,66933457,2016-03-25,62149775,Scott,Peter made me feel more than welcome in his cute house in a very convenient side of Austin. I loved the proximity to the coffee shop as well! Peter treated me like a long-lost cousin in that I never felt like a stranger being tolerated in someone's home! He always answered questions rapidly and even met me at

In [11]:
from pyspark.ml import Pipeline
import sparknlp
sparknlp.start()
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline

pipeline = PretrainedPipeline("analyze_sentiment", lang="en")

analyze_sentiment download started this may take some time.
Approx size to download 4.9 MB
[ | ][OK!]

In [12]:
thousand_row = df.withColumnRenamed("comments", "text")

In [13]:
annotations_df = pipeline.transform(thousand_row)
display(annotations_df)

listing_id id date reviewer_id reviewer_name text document sentence token checked sentiment 3862246 54663524 2015-11-22 985696 Lydia Denise was super friendly and her home was impeccably clean and beyond charming. Her two little dogs were quiet friendly and the separate entrance was very nice. Have never seen such a well stocked bathroom, an keurig right in the bedroom, and excellent water pressure! A great representation and slice of Austin.
Would highly recommend staying with her! List(List(document, 0, 355, Denise was super friendly and her home was impeccably clean and beyond charming. Her two little dogs were quiet friendly and the separate entrance was very nice. Have never seen such a well stocked bathroom, an keurig right in the bedroom, and excellent water pressure! A great representation and slice of Austin.
Would highly recommend staying with her! , Map(sentence -> 0), List())) List(List(document, 0, 79, Denise was super friendly and her home was impeccably clean and beyond charming., Map(sentence -> 0), List()), List(document, 81, 160, Her two little dogs were quiet friendly and the separate entrance was very nice., Map(sentence -> 1), List()), List(document, 162, 268, Have never seen such a well stocked bathroom, an keurig right in the bedroom, and excellent water pressure!, Map(sentence -> 2), List()), List(document, 270, 312, A great representation and slice of Austin., Map(sentence -> 3), List()), List(document, 315, 354, Would highly recommend staying with her!, Map(sentence -> 4), List())) List(List(token, 0, 5, Denise, Map(sentence -> 0), List()), List(token, 7, 9, was, Map(sentence -> 0), List()), List(token, 11, 15, super, Map(sentence -> 0), List()), List(token, 17, 24, friendly, Map(sentence -> 0), List()), List(token, 26, 28, and, Map(sentence -> 0), List()), List(token, 30, 32, her, Map(sentence -> 0), List()), List(token, 34, 37, home, Map(sentence -> 0), List()), List(token, 39, 41, was, Map(sentence -> 0), List()), List(token, 43, 52, impeccably, Map(sentence -> 0), List()), List(token, 54, 58, clean, Map(sentence -> 0), List()), List(token, 60, 62, and, Map(sentence -> 0), List()), List(token, 64, 69, beyond, Map(sentence -> 0), List()), List(token, 71, 78, charming, Map(sentence -> 0), List()), List(token, 79, 79, ., Map(sentence -> 0), List()), List(token, 81, 83, Her, Map(sentence -> 1), List()), List(token, 85, 87, two, Map(sentence -> 1), List()), List(token, 89, 94, little, Map(sentence -> 1), List()), List(token, 96, 99, dogs, Map(sentence -> 1), List()), List(token, 101, 104, were, Map(sentence -> 1), List()), List(token, 106, 110, quiet, Map(sentence -> 1), List()), List(token, 112, 119, friendly, Map(sentence -> 1), List()), List(token, 121, 123, and, Map(sentence -> 1), List()), List(token, 125, 127, the, Map(sentence -> 1), List()), List(token, 129, 136, separate, Map(sentence -> 1), List()), List(token, 138, 145, entrance, Map(sentence -> 1), List()), List(token, 147, 149, was, Map(sentence -> 1), List()), List(token, 151, 154, very, Map(sentence -> 1), List()), List(token, 156, 159, nice, Map(sentence -> 1), List()), List(token, 160, 160, ., Map(sentence -> 1), List()), List(token, 162, 165, Have, Map(sentence -> 2), List()), List(token, 167, 171, never, Map(sentence -> 2), List()), List(token, 173, 176, seen, Map(sentence -> 2), List()), List(token, 178, 181, such, Map(sentence -> 2), List()), List(token, 183, 183, a, Map(sentence -> 2), List()), List(token, 185, 188, well, Map(sentence -> 2), List()), List(token, 190, 196, stocked, Map(sentence -> 2), List()), List(token, 198, 205, bathroom, Map(sentence -> 2), List()), List(token, 206, 206, ,, Map(sentence -> 2), List()), List(token, 208, 209, an, Map(sentence -> 2), List()), List(token, 211, 216, keurig, Map(sentence -> 2), List()), List(token, 218, 222, right, Map(sentence -> 2), List()), List(token, 224, 225, in, Map(sentence -> 2), List()), List(token, 227, 229, the, Map(sentence -> 2), List()), List(token, 231, 237, bedroom,

In [14]:
# return the sentiment with the highest confidence value
@F.udf()
def highestConfidence(sentClassifications):
  idx = np.argmax([x["metadata"]["confidence"] for x in sentClassifications])
  return sentClassifications[idx]["result"]


# return sentiment of the longest sentence
@F.udf()
def longestSentiment(sentClassifications):
  sentLengths = [
      [x["result"], x["end"] - x["begin"]]
    for x in sentClassifications
  ]
  posSent = sum([x[1] for x in sentLengths if x[0] == "positive"])
  negSent = sum([x[1] for x in sentLengths if x[0] == "negative"])
  return "positive" if posSent>negSent else "negative"


# return the overall average sentiment by votes
@F.udf()
def averageSentiment(sentClassifications):
  sentConf = [
      [x["result"], x["metadata"]["confidence"]]
    for x in sentClassifications
  ]
  posSent = np.mean([float(x[1]) for x in sentConf if x[0] == "positive"])
  negSent = np.mean([float(x[1]) for x in sentConf if x[0] == "negative"])
  return "positive" if posSent>=negSent else "negative"


# return the overall average sentiment after normalizing by sentence length
@F.udf()
def classifySentiment(sentClassifications):
  
  sentConf = [
      [x["result"], x["metadata"]["confidence"]]
    for x in sentClassifications
  ]
  posSentConf = np.mean([float(x[1]) for x in sentConf if x[0] == "positive"])
  negSentConf = np.mean([float(x[1]) for x in sentConf if x[0] == "negative"])
  
  sentLengths = [
      [x["result"], x["end"] - x["begin"]]
    for x in sentClassifications
  ]
  posSentLengths = sum([x[1] for x in sentLengths if x[0] == "positive"])
  negSentLengths = sum([x[1] for x in sentLengths if x[0] == "negative"])
  
  # set to NaN if we have null comments
  if posSentLengths == 0 and negSentLengths == 0:
    return np.nan
  
  if abs(posSentLengths - negSentLengths)/(posSentLengths + negSentLengths) > 0.2:
    return "positive" if posSentLengths>negSentLengths else "negative"
  
  else:
    return "positive" if posSentConf>negSentConf else "negative"

In [15]:
# sentiment_df = annotations_df.select("listing_id", "id", "date", "reviewer_id", "reviewer_name", "text", "sentiment",
#                                      averageSentiment("sentiment").alias("avg_sentiment"),
#                                      highestConfidence("sentiment").alias("high_conf_sentiment"),
#                                      longestSentiment("sentiment").alias("long_conf_sentiment"),
#                                      classifySentiment("sentiment").alias("classified_sentiment")
#                                     )
annotations_df.select("listing_id", "id", "date", "reviewer_id", "reviewer_name", "text", "sentiment",
                                     classifySentiment("sentiment").alias("classified_sentiment")
                                    )
display(sentiment_df)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 415.0 failed 1 times, most recent failure: Lost task 0.0 in stage 415.0 (TID 21904, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$dfAssembleNoExtras$1: (string) => array ,embeddings:array >>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$GroupedIterator.takeDestructively(Iterator.scala:1073)
	at scala.collection.Iterator$GroupedIterator.go(Iterator.scala:1089)
	at scala.collection.Iterator$GroupedIterator.fill(Iterator.scala:1127)
	at scala.collection.Iterator$GroupedIterator.hasNext(Iterator.scala:1130)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1334)
	at org.apache.spark.api.python.PythonRDD$.writeIteratorToStream(PythonRDD.scala:368)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.writeIteratorToStream(PythonUDFRunner.scala:50)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread$$anonfun$run$1.apply(PythonRunner.scala:424)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2136)
	at org.apache.spark.api.python.BasePythonRunner$WriterThread.run(PythonRunner.scala:230)
Caused by: java.lang.NullPointerException
	at com.johnsnowlabs.nlp.DocumentAssembler.assemble(DocumentAssembler.scala:98)
	at com.johnsnowlabs.nlp.DocumentAssembler$$anonfun$dfAssembleNoExtras$1.apply(DocumentAssembler.scala:124)
	at com.johnsnowlabs.nlp.DocumentAssembler$$anonfun$dfAssembleNoExtras$1.apply(DocumentAssembler.scala:123)
	... 18 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2350)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2349)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2349)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1102)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1102)
	at scala.Option.foreach(Option.scala:257)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1102)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2581)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2529)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2517)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:897)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2280)
	at org.apache.spark.sql.execution.collect.Collector.runSparkJobs(Collector.scala:270)
	at org.apache.spark.sql.execution.collect.Collector.collect(Collector.scala:280)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:80)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:86)
	at org.apache.spark.sql.execution.ResultCache

In [16]:
avg_sentiment_df = sentiment_df.groupBy("listing_id").pivot("avg_sentiment", ["positive", "negative"]).count().orderBy("listing_id")
avg_sentiment_df = avg_sentiment_df.withColumnRenamed("positive", "avg_sentiment_positive") \
                                   .withColumnRenamed("negative", "avg_sentiment_negative") \
                                   .withColumn("avg_sentiment_positive_percent", F.expr("CASE WHEN avg_sentiment_negative is null and avg_sentiment_positive > 0 THEN 1 WHEN avg_sentiment_positive is null and avg_sentiment_negative > 0 THEN 0 ELSE avg_sentiment_positive/(avg_sentiment_positive + avg_sentiment_negative) END"))
avg_sentiment_df.show()

+----------+----------------------+----------------------+------------------------------+
listing_id|avg_sentiment_positive|avg_sentiment_negative|avg_sentiment_positive_percent|
+----------+----------------------+----------------------+------------------------------+
 2265| 15| 9| 0.625|
 5245| 4| 5| 0.4444444444444444|
 5456| 42| 34| 0.5526315789473685|
 39993| 1| null| 1.0|
 79603| null| 1| 0.0|
 977492| 44| 22| 0.6666666666666666|
 3862246| 64| 29| 0.6881720430107527|
 4030864| null| 1| 0.0|
 7330232| 11| 12| 0.4782608695652174|
 7340115| null| 1| 0.0|
 7340301| null| 6| 0.0|
 7340463| 28| 23| 0.5490196078431373|
 7344751| 2| 2| 0.5|
 7345104| 6| 6| 0.5|
 7348932| 4| 13| 0.23529411764705882|
 7351170| 6| 1| 0.8571428571428571|
 7531639| null| 2| 0.0|
 13361970| 72| 67| 0.5179856115107914|
 13465393| null| 2| 0.0|
 14820133| null| 1| 0.0|
+----------+----------------------+----------------------+------------------------------+
only showing top 20 rows

In [17]:
high_conf_sentiment_df = sentiment_df.groupBy("listing_id").pivot("high_conf_sentiment", ["positive", "negative"]).count().orderBy("listing_id")
high_conf_sentiment_df = high_conf_sentiment_df.withColumnRenamed("positive", "high_conf_sentiment_positive") \
                                               .withColumnRenamed("negative", "high_conf_sentiment_negative") \
                                               .withColumn("high_conf_sentiment_positive_percent", F.expr("CASE WHEN high_conf_sentiment_negative is null and high_conf_sentiment_positive > 0 THEN 1 WHEN high_conf_sentiment_positive is null and high_conf_sentiment_negative > 0 THEN 0 ELSE high_conf_sentiment_positive/(high_conf_sentiment_positive + high_conf_sentiment_negative) END"))
high_conf_sentiment_df.show()

+----------+----------------------------+----------------------------+------------------------------------+
listing_id|high_conf_sentiment_positive|high_conf_sentiment_negative|high_conf_sentiment_positive_percent|
+----------+----------------------------+----------------------------+------------------------------------+
 2265| 17| 7| 0.7083333333333334|
 5245| 7| 2| 0.7777777777777778|
 5456| 58| 18| 0.7631578947368421|
 39993| 1| null| 1.0|
 79603| 1| null| 1.0|
 977492| 54| 12| 0.8181818181818182|
 3862246| 80| 13| 0.8602150537634409|
 4030864| null| 1| 0.0|
 7330232| 21| 2| 0.9130434782608695|
 7340115| 1| null| 1.0|
 7340301| 6| null| 1.0|
 7340463| 37| 14| 0.7254901960784313|
 7344751| 3| 1| 0.75|
 7345104| 10| 2| 0.8333333333333334|
 7348932| 13| 4| 0.7647058823529411|
 7351170| 5| 2| 0.7142857142857143|
 7531639| null| 2| 0.0|
 13361970| 110| 28| 0.7971014492753623|
 13465393| 2| null| 1.0|
 14820133| null| 1| 0.0|
+----------+----------------------------+----------------------------+------------------------------------+
only showing top 20 rows

In [18]:
long_conf_sentiment_df = sentiment_df.groupBy("listing_id").pivot("long_conf_sentiment", ["positive", "negative"]).count().orderBy("listing_id")
long_conf_sentiment_df = long_conf_sentiment_df.withColumnRenamed("positive", "long_conf_sentiment_positive") \
                                               .withColumnRenamed("negative", "long_conf_sentiment_negative") \
                                               .withColumn("long_conf_sentiment_positive_percent", F.expr("CASE WHEN long_conf_sentiment_negative is null and long_conf_sentiment_positive > 0 THEN 1 WHEN long_conf_sentiment_positive is null and long_conf_sentiment_negative > 0 THEN 0 ELSE long_conf_sentiment_positive/(long_conf_sentiment_positive + long_conf_sentiment_negative) END"))
long_conf_sentiment_df.show()

+----------+----------------------------+----------------------------+------------------------------------+
listing_id|long_conf_sentiment_positive|long_conf_sentiment_negative|long_conf_sentiment_positive_percent|
+----------+----------------------------+----------------------------+------------------------------------+
 2265| 15| 9| 0.625|
 5245| 6| 3| 0.6666666666666666|
 5456| 48| 28| 0.631578947368421|
 39993| 1| null| 1.0|
 79603| 1| null| 1.0|
 977492| 43| 23| 0.6515151515151515|
 3862246| 56| 37| 0.6021505376344086|
 4030864| null| 1| 0.0|
 7330232| 14| 9| 0.6086956521739131|
 7340115| 1| null| 1.0|
 7340301| 6| null| 1.0|
 7340463| 35| 16| 0.6862745098039216|
 7344751| 3| 1| 0.75|
 7345104| 8| 4| 0.6666666666666666|
 7348932| 12| 5| 0.7058823529411765|
 7351170| 5| 2| 0.7142857142857143|
 7531639| 2| null| 1.0|
 13361970| 79| 60| 0.5683453237410072|
 13465393| 2| null| 1.0|
 14820133| null| 1| 0.0|
+----------+----------------------------+----------------------------+------------------------------------+
only showing top 20 rows

In [19]:
class_sentiment_df = sentiment_df.groupBy("listing_id").pivot("classified_sentiment", ["positive", "negative"]).count().orderBy("listing_id")
class_sentiment_df = class_sentiment_df.withColumnRenamed("positive", "classified_sentiment_positive") \
                                       .withColumnRenamed("negative", "classified_sentiment_negative") \
                                       .withColumn("classified_sentiment_positive_percent", F.expr("CASE WHEN classified_sentiment_negative is null and classified_sentiment_positive > 0 THEN 1 WHEN classified_sentiment_positive is null and classified_sentiment_negative > 0 THEN 0 ELSE classified_sentiment_positive/(classified_sentiment_positive + classified_sentiment_negative) END"))
class_sentiment_df.show()

+----------+-----------------------------+-----------------------------+-------------------------------------+
listing_id|classified_sentiment_positive|classified_sentiment_negative|classified_sentiment_positive_percent|
+----------+-----------------------------+-----------------------------+-------------------------------------+
 2265| 14| 10| 0.5833333333333334|
 5245| 6| 3| 0.6666666666666666|
 5456| 54| 22| 0.7105263157894737|
 39993| 1| null| 1.0|
 79603| 1| null| 1.0|
 977492| 47| 19| 0.7121212121212122|
 3862246| 62| 31| 0.6666666666666666|
 4030864| null| 1| 0.0|
 7330232| 16| 7| 0.6956521739130435|
 7340115| 1| null| 1.0|
 7340301| 6| null| 1.0|
 7340463| 36| 15| 0.7058823529411765|
 7344751| 2| 2| 0.5|
 7345104| 9| 3| 0.75|
 7348932| 12| 5| 0.7058823529411765|
 7351170| 5| 2| 0.7142857142857143|
 7531639| 1| 1| 0.5|
 13361970| 93| 45| 0.6739130434782609|
 13465393| 2| null| 1.0|
 14820133| null| 1| 0.0|
+----------+-----------------------------+-----------------------------+-------------------------------------+
only showing top 20 rows

In [20]:
all_sentiment_df = avg_sentiment_df.join(high_conf_sentiment_df, ["listing_id"], how="inner") \
                                   .join(long_conf_sentiment_df, ["listing_id"], how="inner") \
                                   .join(class_sentiment_df, ["listing_id"], how="inner")
display(all_sentiment_df)

listing_id,avg_sentiment_positive,avg_sentiment_negative,avg_sentiment_positive_percent,high_conf_sentiment_positive,high_conf_sentiment_negative,high_conf_sentiment_positive_percent,long_conf_sentiment_positive,long_conf_sentiment_negative,long_conf_sentiment_positive_percent,classified_sentiment_positive,classified_sentiment_negative,classified_sentiment_positive_percent
2265,15,9,0.625,17,7,0.7083333333333334,15,9,0.625,14,10,0.5833333333333334
5245,4,5,0.4444444444444444,7,2,0.7777777777777778,6,3,0.6666666666666666,6,3,0.6666666666666666
5456,42,34,0.5526315789473685,58,18,0.7631578947368421,48,28,0.631578947368421,54,22,0.7105263157894737
39993,1,null,1.0,1,null,1.0,1,null,1.0,1,null,1.0
79603,null,1,0.0,1,null,1.0,1,null,1.0,1,null,1.0
977492,44,22,0.6666666666666666,54,12,0.8181818181818182,43,23,0.6515151515151515,47,19,0.7121212121212122
3862246,64,29,0.6881720430107527,80,13,0.8602150537634409,56,37,0.6021505376344086,62,31,0.6666666666666666
4030864,null,1,0.0,null,1,0.0,null,1,0.0,null,1,0.0
7330232,11,12,0.4782608695652174,21,2,0.9130434782608695,14,9,0.6086956521739131,16,7,0.6956521739130435
7340115,null,1,0.0,1,null,1.0,1,null,1.0,1,null,1.0


In [21]:
save_mode = "overwrite"

all_sentiment_df.write \
    .format("com.databricks.spark.csv") \
    .options(header="true") \
    .mode(save_mode) \
    .save("/FileStore/tables/all_sentiment.csv")

In [22]:
display(dbutils.fs.ls("/FileStore/tables"))

path,name,size
dbfs:/FileStore/tables/airlines.csv,airlines.csv,505
dbfs:/FileStore/tables/all_sentiment.csv/,all_sentiment.csv/,0
dbfs:/FileStore/tables/amazon_reviews_us_Digital_Video_Games_v1_00_tsv-5da15.gz,amazon_reviews_us_Digital_Video_Games_v1_00_tsv-5da15.gz,27442648
dbfs:/FileStore/tables/amazon_reviews_us_Software_v1_00_tsv-16dc0.gz,amazon_reviews_us_Software_v1_00_tsv-16dc0.gz,94010685
dbfs:/FileStore/tables/amazon_reviews_us_Video_Games_v1_00_tsv-56dc1.gz,amazon_reviews_us_Video_Games_v1_00_tsv-56dc1.gz,475199894
dbfs:/FileStore/tables/austin_weather_2017.csv,austin_weather_2017.csv,609844
dbfs:/FileStore/tables/calendar.csv,calendar.csv,180039751
dbfs:/FileStore/tables/config.py,config.py,174
dbfs:/FileStore/tables/customer_data.csv/,customer_data.csv/,0
dbfs:/FileStore/tables/data.csv,data.csv,73
